In [ ]:
import pandas as pd
# !pip install pandas_datareader
from pandas_datareader import data as web
from datetime import datetime
from datetime import date

In [ ]:
#ler massa de dados
dreitr = pd.read_csv('DADOS/itr_cia_aberta_DRE_con_2011-2021')
dredfp = pd.read_csv('DADOS/dfp_cia_aberta_DRE_con_2011-2021')

# relação de empresas
empresas = dreitr[['DENOM_CIA','CD_CVM','CNPJ_CIA']].drop_duplicates().set_index('CD_CVM')
# Relação de contas
# contas = empresa[['CD_CONTA','DS_CONTA']].drop_duplicates().set_index('CD_CONTA')
# '3.99.01.01'


In [ ]:
def CalculoLPA(codigo_cvm,conta_cvm='3.99.01.01'):

    #Selecao de empresas nos arquivos principais
    empresa = dreitr[dreitr['CD_CVM']==codigo_cvm]
    empresadfp = dredfp[dredfp['CD_CVM']==codigo_cvm]

    #LPA do ITR
    lpa = empresa[empresa['CD_CONTA'] == conta_cvm]
    lpa = lpa[lpa['ORDEM_EXERC'] == 'ÚLTIMO']
    lpa['DT_INI_EXERC'] = pd.to_datetime(lpa['DT_INI_EXERC'])
    lpa['DT_FIM_EXERC'] = pd.to_datetime(lpa['DT_FIM_EXERC'])
    lpa['Timedelta'] = pd.to_timedelta(lpa['DT_FIM_EXERC']-lpa['DT_INI_EXERC'], unit='D', errors='raise') #calcular o delta dos registros para localizar os registros apenas de trimestres separados e não de acumulado do ano
    lpa = lpa[lpa['Timedelta'] < pd.Timedelta(100, unit='D')] #filtrar somente registros de trimestres em separado
    lpa = lpa[['DT_FIM_EXERC','VL_CONTA']].set_index('DT_FIM_EXERC')

    #LPA do DFP
    lpadfp = empresadfp[empresadfp['ORDEM_EXERC'] == 'ÚLTIMO']
    lpadfp = lpadfp[lpadfp['CD_CONTA'] == conta_cvm]
    lpadfp['DT_INI_EXERC'] = pd.to_datetime(lpadfp['DT_INI_EXERC'])
    lpadfp['DT_FIM_EXERC'] = pd.to_datetime(lpadfp['DT_FIM_EXERC'])
    lpadfp = lpadfp[['DT_FIM_EXERC','VL_CONTA']].set_index('DT_FIM_EXERC')
    lpa['ano'] = lpa.index.year #testar remover linha
    lpadfp['ano'] = lpadfp.index.year


    #Obter lpa somente do quarto tri
    lpaano = lpa.groupby('ano').sum()
    lpaano.rename(columns={'VL_CONTA':'VL_CONTA_AC_3Q'}, inplace=True)
    lpadfpano = lpadfp.groupby('ano').sum()
    lpadfpano.rename(columns={'VL_CONTA':'VL_CONTA_AC_4Q'}, inplace=True)

    lpa4q = lpaano.join(lpadfpano)
    lpa4q['VL_CONTA_SO_4Q'] = lpa4q['VL_CONTA_AC_4Q'] - lpa4q['VL_CONTA_AC_3Q']
    lpa4q.reset_index(inplace=True)
    lpa4q['date'] = lpa4q['ano'].astype(str)+'-12-31'
    lpa4q = lpa4q[['date','ano','VL_CONTA_SO_4Q']].set_index('date')
    lpa4q.rename(columns={'VL_CONTA_SO_4Q':'VL_CONTA'}, inplace=True)

    #Consolidar dataframe com LPAs por trimestre
    lpa_trimestral = pd.concat([lpa, lpa4q]).drop('ano',axis=1)
    lpa_trimestral.index = pd.to_datetime(lpa_trimestral.index)
    lpa_trimestral.sort_index(inplace=True)

    lpa_trimestral['ultimos 4q'] = lpa_trimestral['VL_CONTA'].rolling(4, min_periods=4).sum()
    lpa_trimestral['media 4q'] = lpa_trimestral['VL_CONTA'].rolling(4, min_periods=4).mean()

    lpa_trimestral['CD_CVM'] = codigo_cvm
    lpa_trimestral['CD_CONTA'] = conta_cvm

    if conta_cvm == '3.99.01.01':
        lpa_trimestral['TP_TICKER'] = 'ON'
    elif conta_cvm == '3.99.01.02':
        lpa_trimestral['TP_TICKER'] = 'PN'
    else:
        lpa_trimestral['TP_TICKER'] = ''







    return lpa_trimestral


In [ ]:
CalculoLPA(7617,'3.99.01.02')

In [35]:
!pip freeze requirements.txt




alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.7
anaconda-project==0.8.3
asn1crypto==0.24.0
astroid==2.2.5
astropy==3.2.1
async-generator==1.10
atomicwrites==1.3.0
attrs==21.4.0
Babel==2.7.0
backcall==0.1.0
backports.functools-lru-cache==1.5
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.7.1
bitarray==0.9.3
bkcharts==0.2
bleach==3.1.0
bokeh==1.2.0
boto==2.49.0
Bottleneck==1.2.1
certifi==2019.6.16
cffi==1.15.0
chardet==3.0.4
charset-normalizer==2.0.10
Click==7.0
cloudpickle==1.2.1
clyent==1.2.2
colorama==0.4.1
comtypes==1.1.7
conda==4.7.10
conda-build==3.18.8
conda-package-handling==1.3.11
conda-verify==3.4.2
contextlib2==0.5.5
cryptography==2.7
cycler==0.10.0
Cython==0.29.12
cytoolz==0.10.0
dask==2.1.0
decorator==4.4.0
defusedxml==0.6.0
distributed==2.1.0
docutils==0.14
entrypoints==0.3
et-xmlfile==1.0.1
fastcache==1.1.0
filelock==3.0.12
Flask==1.1.1
fonttools==4.28.5
future==0.17.1


In [ ]:
# gerar todas as planilhas de LPA

# for empr in empresas.index:
#     df = CalculoLPA(empr)
#     df.to_excel('LPA/lpa_'+str(empr)+'.xlsx')

# Salva Relacao de empresas

# empresas.to_excel('AUXILIAR/Empresas_CD_CVM_e_CNPJ.xlsx')

In [ ]:
# dredfp[(dredfp['CD_CONTA']=='3.99.01.02') | (dredfp['CD_CONTA']=='3.99.01.01')][['CD_CONTA','DS_CONTA']].drop_duplicates()